Создаем макет массива с данными о всех клиентах

In [138]:
clients = dict()

Макет массива с данными о всех акциях

In [139]:
stocks = dict()

stocks['sample_ticker'] = {'current_pice' : 5, 'company_name' : '123', 'info' : {}}
stocks['sample_ticker_1'] = {'current_pice' : 5, 'company_name' : '123', 'info' : {}}
stocks

{'sample_ticker': {'current_pice': 5, 'company_name': '123', 'info': {}},
 'sample_ticker_1': {'current_pice': 5, 'company_name': '123', 'info': {}}}

Пишем функцию регистрации нового клиента

In [140]:
def register(client_id):
    global clients
    if client_id not in clients:
        clients[client_id] = {'stocks': {},
                              'registration_date': 0,
                              'history': [],
                              'account': 1000000
                             }

In [165]:
register ('vanko')
clients

{'vanko': {'stocks': {'sample_ticker_1': {'buy_price': 0, 'quantity': 200}},
  'registration_date': 0,
  'history': [],
  'account': 999000}}

Балдеж, идем дальше

Напишем функцию покупки акции


In [175]:
def buy(client_id, ticker, quantity):
    global stocks, clients    
    if ticker not in stocks.keys():                          #возможно while
        print('invalid ticker')
    cost = stocks[ticker]['current_pice'] * quantity
    print('Стоимость сделки: ', cost)
    if cost > clients[client_id]['account']:                 #возможно while
        print('Недостаточно средств')
    else:
        print('Подтвердите сделку, 1 - да, 0 - нет')
        ans = int(input())
        if ans == 1:
            if ticker not in clients[client_id]['stocks'].keys():
                clients[client_id]['stocks'] = {ticker :{'buy_price': 0, 'quantity': 0}}
            clients[client_id]['account'] -= cost
            clients[client_id]['stocks'][ticker]['quantity'] += quantity
            print('Сделка совершена!')
            
            
def sell(client_id, ticker, quantity):
    global stocks, clients    
    if ticker not in stocks.keys():                                   #возможно while
        print('invalid ticker')
    cost = stocks[ticker]['current_pice'] * quantity
    print('Стоимость сделки: ', cost)
    if  quantity > clients[client_id]['stocks'][ticker]['quantity']:              #возможно while
        print('Недостаточно активов')
    else:
        print('Подтвердите сделку, 1 - да, 0 - нет')
        ans = int(input())
        if ans == 1:
            if ticker not in clients[client_id]['stocks'].keys():
                clients[client_id]['stocks'] = {ticker :{'buy_price': 0, 'quantity': 0}}
            clients[client_id]['account'] += cost
            clients[client_id]['stocks'][ticker]['quantity'] += quantity
            print('Сделка совершена!')
        


In [176]:
buy('vanko','sample_ticker_1',50)

Стоимость сделки:  250
Подтвердите сделку, 1 - да, 0 - нет
1
Сделка совершена!


In [179]:
sell('vanko','sample_ticker_1',50)

Стоимость сделки:  250
Подтвердите сделку, 1 - да, 0 - нет
1
Сделка совершена!


In [177]:
stocks

{'sample_ticker': {'current_pice': 5, 'company_name': '123', 'info': {}},
 'sample_ticker_1': {'current_pice': 5, 'company_name': '123', 'info': {}}}

In [180]:
clients

{'vanko': {'stocks': {'sample_ticker_1': {'buy_price': 0, 'quantity': 450}},
  'registration_date': 0,
  'history': [],
  'account': 998250}}